In [1]:
library(polyester)
library(Biostrings)
library(rtracklayer)
library(BSgenome)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The followin

In [2]:
setwd("/Users/tessa/dib-lab/kmerslay-analysis/simulate_reads")

In [3]:
# Start with Quest for Orthologs data
data_dir="/Users/tessa/dib-lab/kmerslay-analysis/QfO_release_2019_04/Eukaryota"
genome_fasta=file.path(data_dir, "UP000005640_9606_DNA.fasta")
addl_fasta=file.path(data_dir, "UP000005640_9606_additional.fasta")
gene_fasta=file.path(data_dir, "UP000005640_9606.fasta")

In [6]:
fasta = readDNAStringSet(genome_fasta)

In [5]:
# test small subset
small_fasta = fasta[1:20]
writeXStringSet(small_fasta, 'test_small.fa')

In [7]:
# let's sim at 20x coverage, uniform:
# ~20x coverage ----> reads per transcript = transcriptlength/readlength * 20
readspertx = round(20 * width(small_fasta) / 100)

In [8]:
# build some fold changes
fold_changes = matrix(c(4,4,rep(1,18),1,1,4,4,rep(1,16)), nrow=20)
head(fold_changes)

4,1
4,1
1,4
1,4
1,1
1,1


In [8]:
# test 20 transcripts
simulate_experiment('test_small.fa', 
                    reads_per_transcript=readspertx,
                    num_reps=c(10,10), 
                    fold_changes=fold_changes, 
                    outdir='timecourse_reads')

### Sim with full QfO dataset

In [9]:
# lets sim using the full dataset, still 20x uniform coverage
# ~20x coverage ----> reads per transcript = transcriptlength/readlength * 20
reads_per_tx_QfO = round(20 * width(fasta) / 100)

In [10]:
fasta

  A DNAStringSet instance of length 20917
         width seq                                          names               
    [1]    462 ATGGGGGCGCCGCTGCTCTCT...GATCCTGTGCTTTACTCTGA tr|A0A024R161|ENS...
    [2]    195 ATGTCCAGCCACGAAGGTGGC...AGAAATCTGGCAAAAAATAA tr|A0A024R1R8|ENS...
    [3]    546 ATGATGAAGTTCAAGCCCAAC...TGCCATCTAGTGTAAGATAG sp|A0A024RBG1|ENS...
    [4]    390 ATGGAGACAGTGGTCACAACT...TCTACATCTGCAGTGCTAGA tr|A0A075B6H5|ENS...
    [5]    348 ATGGAAGCCCCAGCTCAGCTT...AGCAGGATTATAACTTACCT tr|A0A075B6H7|ENS...
    ...    ... ...
[20913]    291 ATGCAGGGGCAAAAGGTGCTC...GCTCCAGAGAGTGTGAATGA tr|V9GZ69|ENSP000...
[20914]    975 ATGGCCCTGCTGGCTCTGGCC...GGTACTGCAGTGGCAACTAA sp|W5XKT8|ENSP000...
[20915]    342 ATGGAGAGTAAATATAAGGAG...TTTCTGGGAATGCATGTTGA sp|W6CW81|AHI87498
[20916]    801 ATGGTGTGTCTGAAGCTCCCT...CAACAGGACTCTTGAGCTGA tr|X5D2U9|ENSP000...
[20917]    384 ATGGGCAGGAAAGAGCATGAA...TGGTCTTGTGGCGCTCCTAG tr|X6R8D5|ENSP000...

In [11]:
#count matrix seems easier than fold changes for this.

# full QfO count matrix
num_timepoints = 2
reads_per_tx_QfO = round(20 * width(fasta) / 100)
countmat = matrix(reads_per_tx_QfO, nrow=length(fasta), ncol=num_timepoints)
                  
# right now, simulated samples (01 and 02) will have identical transcript counts.
# since simulation is random, should be slightly different btwn the samples

In [12]:
head(countmat)

92,92
39,39
109,109
78,78
70,70
71,71


In [ ]:
simulate_experiment_countmat(genome_fasta, readmat=countmat, outdir="QfO_full")

In [13]:
## To add variability, change this count matrix. For example,cbind 20x cov, 50x cov vectors
# reads/tx
reads_per_tx20_QfO = round(20 * width(fasta) / 100) # 20x coverage
reads_per_tx50_QfO = round(50 * width(fasta) / 100) # 50x coverage
counts20_50 <- cbind(reads_per_tx20_QfO, reads_per_tx50_QfO)    

In [14]:
head(counts20_50)

reads_per_tx20_QfO,reads_per_tx50_QfO
92,231
39,98
109,273
78,195
70,174
71,176


In [ ]:
# OR, we can add random (or deterministic) variability
# by scaling countmat by a vector of random (or set) values

## example from tutorial:

#Y = scale(r, X)

# add spikes in expression at certain timepoints to certain transcripts:
#up_early = c(1,2) # transcripts #1 and #2
#up_late = c(3,4) # transcripts # 3 and #4
#countmat[up_early, 1] = 3*countmat[up_early, 1]
#countmat[up_early, 2] = round(1.5*countmat[up_early, 2])
#countmat[up_late, 4] = 6*countmat[up_late, 4]
#countmat[up_late, 5] = round(1.2*countmat[up_late, 5])

In [51]:
simulate_experiment_countmat(genome_fasta, readmat=countmat, outdir="QfO_full") 

In [ ]:
# let's do some full human genome data

In [8]:
hsapiens_fasta="/Users/tessa/dib-lab/kmerslay-analysis/Homo_sapiens/Ensembl/GRCh37/Sequence/WholeGenomeFasta/genome.fa"
hsapiens_gtf="/Users/tessa/dib-lab/kmerslay-analysis/Homo_sapiens/Ensembl/GRCh37/Annotation/Genes/genes.gtf"

## Need to filter the gtf

In [10]:
# import is from rtracklayer
h_gtf <- import(hsapiens_gtf)

In [11]:
h_gtf

GRanges object with 2764635 ranges and 18 metadata columns:
            seqnames            ranges strand |
               <Rle>         <IRanges>  <Rle> |
        [1]        1       11869-12227      + |
        [2]        1       11869-14409      + |
        [3]        1       11872-12227      + |
        [4]        1       11872-14412      + |
        [5]        1       11874-12227      + |
        ...      ...               ...    ... .
  [2764631]        Y 28776794-28776896      - |
  [2764632]        Y 28779492-28779578      - |
  [2764633]        Y 28780670-28780799      - |
  [2764634]        Y 59001391-59001635      + |
  [2764635]        Y 59001391-59001635      + |
                                        source       type     score     phase
                                      <factor>   <factor> <numeric> <integer>
        [1]               processed_transcript       exon      <NA>      <NA>
        [2]               processed_transcript transcript      <NA>      <NA>
    

In [12]:
# first, we want to only keep chromosomes, so we can use the fasta files in the Chromosomes folder of the Ensembl download

In [13]:
seqnames_to_keep = c(1:22, "M", "X", "Y")

In [14]:
#haveNotes <- !is.na(drop(test$Note))
#gene <- drop(gr$Parent)[haveNotes]
filtered <- subset(h_gtf, seqnames  %in% seqnames_to_keep)
export(filtered, "chromosomes.gff3")

In [15]:
filtered

GRanges object with 2529034 ranges and 18 metadata columns:
            seqnames            ranges strand |
               <Rle>         <IRanges>  <Rle> |
        [1]        1       11869-12227      + |
        [2]        1       11869-14409      + |
        [3]        1       11872-12227      + |
        [4]        1       11872-14412      + |
        [5]        1       11874-12227      + |
        ...      ...               ...    ... .
  [2529030]        Y 28776794-28776896      - |
  [2529031]        Y 28779492-28779578      - |
  [2529032]        Y 28780670-28780799      - |
  [2529033]        Y 59001391-59001635      + |
  [2529034]        Y 59001391-59001635      + |
                                        source       type     score     phase
                                      <factor>   <factor> <numeric> <integer>
        [1]               processed_transcript       exon      <NA>      <NA>
        [2]               processed_transcript transcript      <NA>      <NA>
    

In [64]:
# tried making df, which seq_gtf says it accepts. NOPE. Neither of these options work for seq_gtf
hgtf_df <- as.data.frame(filtered)
head(hgtf_df) # this over expands the `attribute` column
gff <- read.delim('chromosomes.gff3', header = FALSE, sep = '\t', skip = 8)
head(gff)

seqnames,start,end,width,strand,source,type,score,phase,exon_id,...,gene_name,gene_source,transcript_id,transcript_name,transcript_source,tss_id,ccds_id,p_id,protein_id,tag
1,11869,12227,359,+,processed_transcript,exon,NA,NA,ENSE00002234944,...,DDX11L1,ensembl_havana,ENST00000456328,DDX11L1-002,havana,TSS15133,NA,NA,NA,NA
1,11869,14409,2541,+,processed_transcript,transcript,NA,NA,NA,...,DDX11L1,ensembl_havana,ENST00000456328,DDX11L1-002,havana,TSS15133,NA,NA,NA,NA
1,11872,12227,356,+,transcribed_unprocessed_pseudogene,exon,NA,NA,ENSE00002234632,...,DDX11L1,ensembl_havana,ENST00000515242,DDX11L1-201,ensembl,TSS178604,NA,NA,NA,NA
1,11872,14412,2541,+,transcribed_unprocessed_pseudogene,transcript,NA,NA,NA,...,DDX11L1,ensembl_havana,ENST00000515242,DDX11L1-201,ensembl,TSS178604,NA,NA,NA,NA
1,11874,12227,354,+,transcribed_unprocessed_pseudogene,exon,NA,NA,ENSE00002269724,...,DDX11L1,ensembl_havana,ENST00000518655,DDX11L1-202,ensembl,TSS192941,NA,NA,NA,NA
1,11874,14409,2536,+,transcribed_unprocessed_pseudogene,transcript,NA,NA,NA,...,DDX11L1,ensembl_havana,ENST00000518655,DDX11L1-202,ensembl,TSS192941,NA,NA,NA,NA


V1,V2,V3,V4,V5,V6,V7,V8,V9
1,transcribed_unprocessed_pseudogene,transcript,11874,14409,.,+,.,gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000518655;transcript_name=DDX11L1-202;transcript_source=ensembl;tss_id=TSS192941;
1,transcribed_unprocessed_pseudogene,exon,12010,12057,.,+,.,exon_id=ENSE00001948541;exon_number=1;gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000450305;transcript_name=DDX11L1-001;transcript_source=havana;tss_id=TSS138561;
1,transcribed_unprocessed_pseudogene,transcript,12010,13670,.,+,.,gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000450305;transcript_name=DDX11L1-001;transcript_source=havana;tss_id=TSS138561;
1,transcribed_unprocessed_pseudogene,exon,12179,12227,.,+,.,exon_id=ENSE00001671638;exon_number=2;gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000450305;transcript_name=DDX11L1-001;transcript_source=havana;tss_id=TSS138561;
1,transcribed_unprocessed_pseudogene,exon,12595,12721,.,+,.,exon_id=ENSE00002270865;exon_number=2;gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000518655;transcript_name=DDX11L1-202;transcript_source=ensembl;tss_id=TSS192941;
1,transcribed_unprocessed_pseudogene,exon,12613,12697,.,+,.,exon_id=ENSE00001758273;exon_number=3;gene_biotype=pseudogene;gene_id=ENSG00000223972;gene_name=DDX11L1;gene_source=ensembl_havana;transcript_id=ENST00000450305;transcript_name=DDX11L1-001;transcript_source=havana;tss_id=TSS138561;


## Subset fasta instead

In [16]:
cdkn2a_ranges <- subset(filtered, gene_name == "CDKN2A")

In [29]:
cdkn2a_ranges
export(cdkn2a_ranges, "cdkn2a.gff3")

GRanges object with 153 ranges and 18 metadata columns:
        seqnames            ranges strand |               source       type
           <Rle>         <IRanges>  <Rle> |             <factor>   <factor>
    [1]        9 21967751-21968241      - |       protein_coding        UTR
    [2]        9 21967751-21968241      - |       protein_coding       exon
    [3]        9 21967751-21994490      - |       protein_coding transcript
    [4]        9 21967752-21968227      - |       protein_coding        UTR
    [5]        9 21967752-21968241      - |       protein_coding        UTR
    ...      ...               ...    ... .                  ...        ...
  [149]        9 21994820-21995268      - |       protein_coding        UTR
  [150]        9 21994820-21995300      - |       protein_coding        UTR
  [151]        9 21994820-21994872      - | processed_transcript       exon
  [152]        9 21994820-21995268      - |       protein_coding       exon
  [153]        9 21994820-219953

In [18]:
hsap_fasta = readDNAStringSet(hsapiens_fasta)

In [27]:
seqinfo(hsap_fasta)

Seqinfo object with 25 sequences from an unspecified genome:
  seqnames seqlengths isCircular genome
  1         249250621       <NA>   <NA>
  2         243199373       <NA>   <NA>
  3         198022430       <NA>   <NA>
  4         191154276       <NA>   <NA>
  5         180915260       <NA>   <NA>
  ...             ...        ...    ...
  21         48129895       <NA>   <NA>
  22         51304566       <NA>   <NA>
  X         155270560       <NA>   <NA>
  Y          59373566       <NA>   <NA>
  MT            16569       <NA>   <NA>

In [19]:
x <- DNAStringSet(c(chr1="ACAANAAGG", chr2="GGGGTTT"))
getSeq(x, GRanges(c("chr2:4-7:-", "chr1:2-7")))


  A DNAStringSet instance of length 2
    width seq
[1]     4 AAAC
[2]     6 CAANAA

In [22]:
ranges(cdkn2a_ranges)


IRanges object with 153 ranges and 0 metadata columns:
            start       end     width
        <integer> <integer> <integer>
    [1]  21967751  21968241       491
    [2]  21967751  21968241       491
    [3]  21967751  21994490     26740
    [4]  21967752  21968227       476
    [5]  21967752  21968241       490
    ...       ...       ...       ...
  [149]  21994820  21995268       449
  [150]  21994820  21995300       481
  [151]  21994820  21994872        53
  [152]  21994820  21995268       449
  [153]  21994820  21995300       481

In [28]:
getSeq(hsap_fasta, cdkn2a_ranges)

ERROR: Error: subscript contains invalid names


In [32]:
export(subset(filtered, gene_name == "GNAS"), "GNAS.gff3")

export(subset(filtered, gene_name == "AGER"), "AGER.gff3")

export(subset(filtered, gene_name == "CSF2RA"), "CSF2RA.gff3")

export(subset(filtered, gene_name == "GNRHR"), "GNRHR.gff3")

export(subset(filtered, gene_name == "XBP1"), "XBP1.gff3")

In [ ]:
## OK, gff3/gtf does NOT work rn in polyester. instead of using R, let's use bedtools
#bedtools getfasta -fi ../Homo_sapiens/Ensembl/GRCh37/Sequence/WholeGenomeFasta/genome.fa  -bed gnas.gff3 > GNAS.fasta

  A DNAStringSet instance of length 6
        width seq                                           names               
[1] 249250621 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 1
[2] 243199373 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 2
[3] 198022430 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 3
[4] 191154276 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 4
[5] 180915260 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 5
[6] 171115067 NNNNNNNNNNNNNNNNNNNNN...NNNNNNNNNNNNNNNNNNNNN 6

In [63]:
# none of seq_gtf or simulate experiment with gtf file work, even using the filtered gff file. INSTEAD, subset the fasta first using the gff, then feed the specific fasta file into polyester

#seq_gtf(gtf=gff, seqs=hsap_fasta, feature = "transcript", exononly = TRUE, idfield = "transcript_id", attrsep = "; ")
#seq_gtf(filtered, seqs=hsap_fasta, feature = "transcript", exononly = TRUE, idfield = "transcript_id", attrsep = "; ")

ERROR: Error in seq_gtf(gtf = gff, seqs = hsap_fasta, feature = "transcript", : one or more columns of gtf have the wrong class


In [7]:
# OK THIS ACTUALLY WORKS --> can do this for each gene
#setwd("/Users/tessa/dib-lab/kmerslay-analysis/simulate_reads")
gnas = readDNAStringSet("GNAS.fasta")
outD= "hsapiens_sim/gnas"
num_timepoints = 2
#reads_per_tx_gnas = round(20 * width(gnas) / 100)
reads_per_tx = 1000
countmat = matrix(reads_per_tx, nrow=length(gnas), ncol=num_timepoints)

simulate_experiment_countmat(fasta="GNAS.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir=outD)

In [11]:
# use this for all simulations: 5 samples, 1000 reads per transcript
num_timepoints = 5
reads_per_tx = 1000

In [12]:
#all genes
setwd("/Users/tessa/dib-lab/kmerslay-analysis/simulate_reads")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("AGER.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="AGER.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/AGER")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("CDKN2A.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="CDKN2A.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/CDKN2A")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("CSF2RA.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="CSF2RA.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/CSF2RA")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("GNAS.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="GNAS.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/GNAS")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("GNRHR.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="GNRHR.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/GNRHR")

countmat = matrix(reads_per_tx, nrow=length(readDNAStringSet("XBP1.fasta")), ncol=num_timepoints)
simulate_experiment_countmat(fasta="XBP1.fasta", readmat=countmat, reportCoverage=TRUE, readlen=150, paired=FALSE, outdir="hsapiens_sim/XBP1")




[1] "DNAStringSet"
attr(,"package")
[1] "Biostrings"

In [38]:
??simulate_experiment

R Information

Help files with alias or concept or title matching
‘simulate_experiment’ using regular expression matching:


polyester::simulate_experiment
                        simulate RNA-seq experiment using negative
                        binomial model
  Aliases: simulate_experiment
polyester::simulate_experiment_countmat
                        Simulate RNA-seq experiment
  Aliases: simulate_experiment_countmat
polyester::simulate_experiment_empirical
                        Simulate RNA-seq experiment based on abundances
                        from a data set
  Aliases: simulate_experiment_empirical


Type '?PKG::FOO' to inspect entries 'PKG::FOO', or 'TYPE?PKG::FOO' for
entries like 'PKG::FOO-TYPE'.




In [52]:
# simulate coding reads (exon only) using gtf
simulate_experiment(#fasta= hsapiens_fasta,
                    #gtf = hsapiens_gtf,
                    #gtf = "/Users/tessa/dib-lab/kmerslay-analysis/Homo_sapiens/Ensembl/GRCh37/Annotation/Genes/genes_head200.gtf",
                    gtf = "chromosomes.gff3",
                    seqpath = "/Users/tessa/dib-lab/kmerslay-analysis/Homo_sapiens/Ensembl/GRCh37/Sequence/Chromosomes",
                    #meanmodel=TRUE, # draw baseline mean count numbers from a model based on transcript length.
                    outdir='hsapiens_sim',
                    idfield = "transcript_id",
                    reads_per_transcript = 300,
                    #gzip =TRUE,
                    seed =142,
                    #exononly=TRUE,
                    #feature = "transcript"
                    )
                    #reads_per_transcript = 300,
                    #reads_per_transcript=readspertx,
                    #readmat=countmat_hsapiens, 
                    #num_reps=c(10,10), 

# Reads can either be simulated from a FASTA file of transcripts (provided with the fasta argument)
# OR from a GTF file plus DNA sequences (provided with the gtf and seqpath arguments). Simulat- ing from a GTF file and DNA sequences may be a bit slower: it took about 6 minutes to parse the GTF/sequence files for chromosomes 1-22, X, and Y in hg19.

parsing gtf and sequences...


ERROR: Error in if (substr(names(these_seqs)[1], 1, 1) == "\"") {: missing value where TRUE/FALSE needed


In [ ]:
# use simulate_experiment_empirical to model counts after real data
#simulate_experiment_empirical(hsapiens_fasta, grouplabels=pData(bg)$group, gtf=hsapiens_gtf,
#    seqpath=chr22seq, mean_rps=5000, outdir='empirical_reads', seed=1247)

Changing other parameters and adding bias
The following parameters can be provided to simulate_experiment and simulate_experiment_countmat:

- **readlen**: Read length (default 100)
- paired: Whether the reads should be paired-end (default TRUE)
- distr: Distribution from which to draw the fragment lengths. Default is 'normal', mean=250 and sd=25. Other options are 'empirical' and 'custom'. 'empirical' means fragment lengths are drawn from a length distribution we estimated from real data, and 'custom' requires you to provide a logspline density object from which you'd like to draw the fragment lengths.
- **fraglen**: The mean fragment length, if using a normal distribution (default 250)
- fragsd: Standard devation of fragment lengths, if using a normal distribution (default 25)
- **error_model**: How should sequencing errors be simulated? The default is that sequencing errors are uniformly distributed across samples, reads, and nucleotides, at an error rate of 0.5%. Other options are 'illumina4', 'illumina5', and 'custom', where the Illumina error models were estimated from a real data set and ship with GemSIM (McElroy, Luciani, and Thomas 2012), and 'custom' allows you to use GemSIM to estimate an error model from your data set. See ?add_platform_error and/or the GemSIM paper for details. Code we used to modify GemSIM's Illumina error models for Polyester are available at our GitHub repository.
- **error_rate**: In the uniform error model, probability that the sequencer records the wrong nucleotide at any given base (default 0.005).
- **bias**: Positional bias model to use when fragmenting transcripts. By default, all fragments from a transcript are equally likely ('none'). Other choices are 'rnaf' and 'cdnaf', which mimic positional bias arising from different fragmentation protocols. See ?generate_fragments and our manuscript (Frazee et al, 2014) for details.
- **gc_bias**: sample-specific GC bias models to be used to change expression values after read numbers are assigned. We modeled transcript expression as a function of GC content for 7 biological replicates in a real data set, and shift expression values accordingly. See ?add_gc_bias for details. Ignored in simulate_experiment_countmat.
- frag_GC_bias: A sample-specific GC content bias on the fragment level instead of the transcript level. See ?simulate_experiment
- strand_specific: Whether the experiment should be strand-specific or not, default is FALSE so an unstranded experiment.
- meanmodel: Set to TRUE to estimate reads_per_transcript as a data-driven function of transcript length. Ignored in simulate_experiment_countmat.
- lib_sizes: multiplicative library size factor for each replicate in the experiment. Ignored in simulate_experiment_countmat.

# Using real data to guide simulation
We also provide a function simulate_experiment_empirical that takes a real transcript expression matrix (in FPKM or RPKM units) and corresponding annotation to simulate an experiment with abundances and differential expression fold changes similar to those given in the expression matrix. This function is compatible with the Ballgown package, or you can simply provide a transcript-by-replicate expression matrix generated with your favorite abundance estimation software.